# WHIN Supply Chain Web Scraper 

### Team Members 
1. Deepika Jindal 
2. Mihir Bhatia 

## Importing Packages 

In [1]:
import urllib.request
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urltools
import time 
from pandas import ExcelWriter
from pandas import ExcelFile
import time

## Get base URL of the website 

This function lets you extract the base URL of the website. It will be used to check if a web page is from the original webiste that we intended to scrape. 

In [2]:
def get_base_url(url):
    from urllib.parse import urlsplit
    base_url = "{0.scheme}://{0.netloc}/".format(urlsplit(url))
    

    if base_url.startswith("https"):
        base_url = base_url.replace('https://','')
    elif base_url.startswith("http"):
        base_url = base_url.replace('http://','')
        
    if base_url.endswith('/'):
        base_url = base_url.replace('/','')
    return base_url


##  Does the page exist ? 
Our web scrapper will go the website and use the http protocol to check if the web page which we intend to scrape exists 

In [3]:
def does_page_exist(url):
    exists = 0 
    try:
        request = requests.get(url)
        if request.status_code == 200:
            exists = 1 
            
    except: 
        pass
    
    return exists



In [4]:
# Function to extract links from a URL------------------------------------------
def href_scrapper(url):

    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        a_tag = soup.find_all('a')
        img_tag = soup.find_all('img')
        link_tag = soup.find_all('link')
        link_list = list()

        for links in a_tag:
            link_list.append(links.get('href'))

        for links in link_tag:
            link_list.append(links.get('href'))

        for links in img_tag:
            link_list.append(links.get('src'))

        return link_list

    except:
        return 0

## What if the link extracted has no http in it ? 
There will be cases in which the link you extracted has no http in the beginning. This function adds an http to the beginning of the link to keep all links similar in structure. This will be useful during string manipulations and will standardize link structure 

In [5]:
# Function to create complete links---------------------------------------------
def link_former(url):
    x = url

    if x.startswith("http"):
        return x

    else:
        full_url = original_url + x
        return full_url



## Save files 
This function saves files with various formats. Our main web scrapper doesn't use all these extensions. But we provide this function incase somoene wants to scrape images, PDFs and webpages 

In [6]:
# Function to save links to directory of python file----------------------------
def file_save(url, i):
    save_error = 0
    full_url = url
    try:
        if str(full_url).endswith(".htm"):
            name = "WebPage" + str(i) + ".htm"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".pdf"):
            name = "Pdf" + str(i) + ".pdf"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".jpg"):
            name = "Image" + str(i) + ".jpg"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".jpeg"):
            name = "Image" + str(i) + ".jpeg"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".png"):
            name = "Image" + str(i) + ".png"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".css"):
            name = "CSS File" + str(i) + ".css"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        else:
            name = "WebPage" + str(i) + ".html"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        return m
    except:
        save_error = save_error + 1



## Now it's time to crawl
The crawl function takes a link, with a number ID and output file and outputs all the daughter links associated with that parent link. 

In [7]:
'''
crawl function = takes link as input and crawls relevant webpages from the parent 

Inputs 
----------

Outputs
----------
'''
def crawl(original_url, num_Id, output_file):
    
    # Accepting input of URL and depth----------------------------------------------
    tic = time.time()
    
    parent_list = [original_url]
    url_to_check = get_base_url(original_url)
    print(url_to_check)
    url_to_check = str(url_to_check)

    layer_stop = 1
    layer = 0


    #initializing all the requiered lists---------------------------------------------
    visited_all  = [original_url]
    visited_current_layer = []
    child_list =[]
    child_list_filtered = [original_url]
    #columns = ['Link','Parent Link', 'Layer']
    df = pd.DataFrame()

    Di={}

    # Main execution of scrapper----------------------------------------------------

    #looping through layers 
    while layer < layer_stop:
    
        #looping through URLs in parent-list
        for url in parent_list:
        
            #scraping the children from the parent url----------------------------
            if href_scrapper(url) != 0:
                child_list = href_scrapper(url)
    
            
            for child in child_list:
                if child != None:
                    ch=child
                    #if child link is of the form "index.php/blahblah" and parent ends with '/'
                    #---> "parentlink/index.php/blahblah"
                    if child.startswith('/'):
                        child= str(url) + str(child)
                    
                    if url.endswith('/') and url_to_check not in child:
                        child = str(url) + str(child)
                    
                #normalize the child links-------------------------------------
                    child=urltools.normalize(child)  
                    

                    #filtering out based on 1) External 2) Repeating 3) Invalid links---------------------------
                    if url_to_check in child and child not in visited_all and does_page_exist(child)==1 and ch not in Di:
                        child_list_filtered.append(child)
                        Di[ch]=1
                
                    #adding everthing to visited all--------------------
                    if child not in visited_all:
                        child_slash = child + '/'
                        visited_all.append(child)  
                        visited_all.append(child_slash)
                    
                    
            #adding  the visited and filtered children into the "current visited layer" ------ 
            for child_filtered in child_list_filtered:
                visited_current_layer.append(child_filtered)

            #creating a Pandas dataframe to store everything for download----------
            layer_number = [layer+1]*len(child_list_filtered)
            parent_of_child = [url]*len(child_list_filtered)

            df_child = pd.DataFrame(child_list_filtered)
            df_parent = pd.DataFrame(parent_of_child)
            df_layer = pd.DataFrame(layer_number)


            df_to_be_added = pd.concat([df_child,df_parent,df_layer], axis=1)
            df = pd.concat([df,df_to_be_added],ignore_index=True, axis = 0)
            #----------------------------------------------------------------------
        
            #emptying the child lists
            child_list = []
            child_list_filtered = []
        
        #condition to stop filtering-----------------------------------------------
        if not visited_current_layer :
            layer_stop = layer_stop 
        else:
            layer_stop += 1

    
        #child layer is now parent layer--------------------------------------------
        parent_list = []
    
        #we only dont add .png, .jpg , .pdf to the new parent layer 
        for visited_current in visited_current_layer: 
            print(visited_current)
            if(not visited_current.endswith('.png') and not visited_current.endswith('.jpg') and not  visited_current.endswith('.pdf')):
                parent_list.append(visited_current)
            
            
        #displaying the links in different layers----------------------------------
        #print("Links in LAYER:" + str(layer+1))
        print("No of links = " + str(len(visited_current_layer)))
        #print(visited_current_layer)
        print("\n")
        visited_current_layer = [] 
        #updating the layer number
        layer +=1
    df.to_csv(output_file + '/' + str(num_Id) + '_' + str(url_to_check) +  '.csv', sep=',', encoding='utf-8')
    return df, num_Id

## Function to mail me when scraping or crawling is done 

In [ ]:
import smtplib
 
def sendemail(from_addr, to_addr_list, cc_addr_list,
              subject, message,
              login, password,
              smtpserver='smtp.gmail.com:587'):
    header  = 'From: %s' % from_addr
    header += 'To: %s' % ','.join(to_addr_list)
    header += 'Cc: %s' % ','.join(cc_addr_list)
    header += 'Subject: %s' % subject
    message = header + message
 
    server = smtplib.SMTP(smtpserver)
    server.starttls()
    server.login(login,password)
    problems = server.sendmail(from_addr, to_addr_list, message)
    server.quit()

## Functions that will be used to scrape the crawled webpages 

In [8]:
'''
Functions that will be used by the scraper function 
tag_visible = 
text_from_html = 
'''

import requests
page = requests.get("https://docs.python.org/3/howto/urllib2.html")
from bs4 import BeautifulSoup
from bs4.element import Comment
soup = BeautifulSoup(page.content, 'html.parser')


#information based on:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True
p_tag=soup.find_all('p')

#extract text from html: 
def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)
web_text = []

## Scrape the webpages 
This function allows us to scrape all the webpages inside the dataframe that contains the crawled links

In [10]:
import json
import urllib
import os 
extensions=('.png', '.jpg','.jpeg','.pdf', '.gif','.ico')
def scrape(df, scraped_website_path, website_index, website_name):
    TempD={}
    for index,item in df.iterrows():
        #print(item[1],item[0])
        try:      
            #for i in extensions:
            #print(item[1])
            if item[1].endswith(extensions):
                print(item[1],"skipped 1")
                print(" ")
                
            elif ".gif&" in item[1]:
                print(item[1],"skipped 2")
                print(" ")
                
            elif "facebook.com" in item[1] or "linkedin.com" in item[1] or "twitter.com" in item[1]:
                print(item[1], "skipped because social network link")
                print(" ")
                
            else:
                print(item[1])
                print(" ")
                html = urllib.request.urlopen(item[1]).read()
                #web_text.append(text_from_html(html))
                print(text_from_html(html))
                time.sleep(3)
                #save_url(str(item[1]), index)
                TempD[item[1]]=text_from_html(html)
        #label: end
        except: 
            pass
    with open(scraped_website_path + '/' + 'json_' + str(website_index) + str(get_base_url(website_name)) +  '.json', 'w') as outfile:  
        json.dump(TempD, outfile)
        
    return 1

## Crawler - that works with an excel sheet containing all the relevant webpages 

In [11]:

# web_crawler_all() = Scraper for multiple websites# web_cr 
'''
Inputs
-----------
excel_path = Excel file path which contains list of websites to be crawled  
sheet = sheet name of this excel file 
web_col = the column no of in which the website hompages are located 
index_col = the column with indices of websites
parent_col = the column with the parent names
output_folder = path of the output folder where we wish to save the websites 
from_website = the index no. from website (typically starting from 1)
to_website = the index no of the website till we want to continue scraping (scraped websites will include the 
                websites with this index)

Output 
-----------
CSV files containing a list of child links for each website are saved into the output_folder path 

'''
def web_crawler_all(excel_path, sheet, web_col, index_col, parent_col,  output_folder, from_website, to_website):
    df = pd.read_excel(excel_path , sheet_name=sheet)
    df = df.iloc[from_website-1 : to_website, :]
    print(df)
    for index,row in df.iterrows():
        num_Id=int(row[index_col])
        original_url=row[web_col]

        
        try:
            websites= pd.read_csv(output_folder + str(int(num_Id)) + '_' + str(original_url) + '.csv')
            print("website is already crawled")
        except:
            websites, num_Id = crawl(original_url ,num_Id ,output_folder)
            
        
    print("All links from the websites indicated have been crawled")
    print("Output File :  " + str(output_folder))
    print("No. of websites scraped : " + str(to_website - from_website + 1))
    return 1

### Things to be added
- index to track what has been scrolled 
- time taken by the href function
- file structure for all webpages


## Scraper that works with a file containing all the scrapped webpages in an excel sheet

In [12]:
'''
Inputs 
----------
excel_path = Excel file path which contains list of websites to be crawled  
sheet = sheet name of this excel file 
crawled_website_path = path of file where all the crawled websites are stored  
scraped_website_path = path of file where all the scraped websites are stored
from_website - to_website = website index number from-to which we want to scrape and store in JSON 
web_col = column number of websites (0,1,2....)

Output
----------
JSON files for each parent website in scraped_website_path folder. The path of the scraped websites 
is the same as that crawled websites 
'''

def web_scraper_all(df, crawled_website_path, scraped_website_path, from_website, to_website,web_col):
    #get excel sheet containing websites 
  #  df = pd.read_excel(excel_path , sheet_name=sheet)
    df = df.iloc[from_website-1 : to_website, :]   
    
    #loop over all the websites and extract child links. Store them in JSON files 
    
    for index, rows in df.iterrows():
        #setting the website index and name 
        website_index = int(rows[0] )
        website_name = rows[web_col]
        
        #generating filename and getting the csv file of child links 
        filename = crawled_website_path + '/'+ str(website_index)+ '_' + str(get_base_url(website_name)) + ".csv"
        df_of_child_links = pd.read_csv(filename)
        print(filename,df_of_child_links )
        #executing the scrape function 
        flag = scrape(df_of_child_links , scraped_website_path, website_index, website_name) 
        
        #checkpoint to see if scraping is complete 
        if flag == 1 :
            print(website_name + ' has been scraped successfully')
            #print('scraping percentage complete = ' + str((i/to_website)*100) + '%')
            flag = 0

## Excection of the scrapper_all function 

In [16]:
#executing the scraper function 

excel_path = 'batchwise_companies.xlsx'
sheet = 'batch4'
df = pd.read_excel(excel_path , sheet_name=sheet)
crawled_website_path = 'Crawled'
scraped_website_path = 'Scraped'
from_website = 1
to_website = 5
web_col = 2 
df = pd.read_excel(excel_path , sheet_name=sheet)
#print(df)
web_scraper_all(df, crawled_website_path, scraped_website_path, from_website, to_website,web_col)

#send mail 
sendemail(from_addr = 'krannertdcmme@gmail.com', to_addr_list = ['mihirbhatia999@gmail.com'],
          cc_addr_list = ['djindal@purdue.edu'],subject = 'Test mail',message = 'this is a test mail 2 using Python',
          login = 'krannertdcmme', password = 'Abc12345!', smtpserver = 'smtp.gmail.com:587')

Crawled/2_carmeleng.com.csv     Unnamed: 0                                                  0  \
0            0                               http://carmeleng.com   
1            1                          http://www.carmeleng.com/   
2            2               http://www.carmeleng.com/default.asp   
3            3                   http://carmeleng.com/favicon.ico   
4            4            http://www.carmeleng.com/contact_us.asp   
5            5            http://www.carmeleng.com/newsletter.asp   
6            6  http://www.carmeleng.com/http:/www.gobuckaroo.com   
7            7           http://www.carmeleng.com/carmelstyle.css   
8            8  http://www.carmeleng.com/images/CE_SplashPage-...   
9            9  http://www.carmeleng.com/images/CE_SplashPage-...   
10          10  http://www.carmeleng.com/images/CE_SplashPage-...   
11          11          http://www.carmeleng.com/images/white.gif   
12          12  http://www.carmeleng.com/images/CE_SplashPage-...   
13    

http://carmeleng.com/favicon.ico skipped 1
 
http://www.carmeleng.com/contact_us.asp
 
                      Contact Us       Carmel Engineering, Inc. 413 East Madison Street P.O. Box 67 Kirklin, IN 46050  Phone: 1 (765) 279-8955  Toll Free: 1 (888) 472-0497  Fax: 1 (765) 279-8966 Request further information at:  sales@carmeleng.com   Randy Weaver President  rweaver@carmeleng.com   Allan Roden Product & Process Engineer  aroden@carmeleng.com           For additional information, please fill out the following form:         *First Name      *Last Name      Company      Phone Number      *Email      Service Type    -- Select a Reason -- Custom Machine Design and Build Manufacturing Line - Process Planning Part/Equipment Replacement Part/Equipment Refurbishment Research & Development General Machining        This form protected by WebSpamProtect . JavaScript must be enabled in your browser to view this image.    Please type the code shown in the image:      (*) Required fields             

          Site Map  Site Credits  Home    About  News  Products  Service  Contact         Whether you’re building smaller HP locomotives to do big jobs or making higher HP locomotives more efficient, Dayton-Phoenix can help.  Learn More         Next      For well over half a century, Dayton-Phoenix Group has supplied quality electrical and locomotive components to the industrial and railroad markets.    DPG Updates   Dayton-Phoenix Group Announces New Advantage Services™ for GP 38, GP40 and SD 40 Locomotives  06-07-18  Read the Full Article    Dayton-Phoenix Group Announces New Advantage Series™ Rooftop AC Unit and Phase Out of Older Rooftop AC Unit  05-03-18  Read the Full Article    DPG to Attend 2018 ASLRRA CONNECTIONS  01-12-18  Read the Full Article    Dayton-Phoenix Group Achieves ISO 9001:2015 Certification  12-13-17  Read the Full Article      Toilets   Our toilets are a compact and efficient facility for crew convenience that doesn’t disturb the crew environment.  Learn More  

https://www.dayton-phoenix.com/siteCredits.php
 
          Site Map  Site Credits  Home    About  News  Products  Service  Contact       Technical Resources   Debian GNU/Linux  Apache HTTP Server  PHP Scripting Language  MySQL Database Server   Standards Guidelines   Cascading Style Sheets  XHTML 1.0  Microformats     Site Credits  About This Website  Dayton-Phoenix Group is committed to making dayton-phoenix.com a valuable resource for our customers. To that end, this website has been built using the latest web standards allowing for the widest audience possible. We hope that you find it usable.  Feedback  We are always interested in feedback regarding this website. Please direct any comments or concerns to webmaster@dayton-phoenix.com .               © 2018 Dayton-Phoenix Group | All Rights Reserved         
https://www.dayton-phoenix.com/news.php
 
          Site Map  Site Credits  Home    About  News  Products  Service  Contact       News   Dayton-Phoenix Group Announces New Advant

https://www.dayton-phoenix.com/index.php?dt=PL
 
          Site Map  Site Credits  Home    About  News  Products  Service  Contact         As commuting costs rise worldwide, an increasing range of hybrid locomotive designs are emerging in the market, combining the best of both freight and passenger products.  Learn More         Next      For well over half a century, Dayton-Phoenix Group has supplied quality electrical and locomotive components to the industrial and railroad markets.    DPG Updates   Dayton-Phoenix Group Announces New Advantage Services™ for GP 38, GP40 and SD 40 Locomotives  06-07-18  Read the Full Article    Dayton-Phoenix Group Announces New Advantage Series™ Rooftop AC Unit and Phase Out of Older Rooftop AC Unit  05-03-18  Read the Full Article    DPG to Attend 2018 ASLRRA CONNECTIONS  01-12-18  Read the Full Article    Dayton-Phoenix Group Achieves ISO 9001:2015 Certification  12-13-17  Read the Full Article      Toilets   Our toilets are a compact and efficient f

...

https://www.dayton-phoenix.com/dynamic_supplier_manual/VR&R.XLS
 


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


 =    ;  ?  Verified By:¾  ; 	 = = >  × 0 N  ¤z z z =[^z z z =[^°Æ Ï � 
 
 '   ] 8       	 «. H 	 
 �	 «. ` 	 
. `             ] *      �	 «. H 	 
. H             ] 8         ') T   S) Ì         ğÊÛ#                 ] *       �  ') T   S) T             ] *     ! �  ') x   S) Ì            ] *     " �  ') x   S) Ì             ] *     #   ™* `   Å* `             ] 8 	     $   '# T   S# Ì         ¸ËÛ(                 ] * 
    (   ™$ `   Å$ `             ] 8      )   ' T   S Ì         €ÌÛ-                 ] *     * �  ' T   S T             ] *     + �  ' x   S Ì            ] *     , �  ' x   S Ì             ] *     -   ™ `   Å `             ] :     .  h ‡  Å ‡        A@    Q Rectangle 46] *     /  ä  '  ä               ] 8      ?  " ,   v , „         ÀÍÛ@                 ] *     ; � " , 0  v , 0    

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://www.dayton-phoenix.com/dynamic_supplier_manual/west_lafayette_facility_color_small.jpg skipped 1
 
https://www.dayton-phoenix.com/dynamic_supplier_manual/z_navicon_top.gif skipped 1
 
https://www.dayton-phoenix.com/dynamic_supplier_manual/z_navicon_overview.gif skipped 1
 
https://www.dayton-phoenix.com has been scraped successfully
Crawled/4_dyna-fab.org.csv    Unnamed: 0                    0                  0.1  0.2
0           0  http://dyna-fab.org  http://dyna-fab.org    1
http://dyna-fab.org
 
                                                                                                                                                                 DynaFab Corp. has been providing solutions to 
								our customer manufacturing needs for 30 years. 
								Dyna-Fab’s large tonnage presses for metal 
								stamping and drawing provide capabilities 
								generally found only in OEM manufactures. 
								Coupled with a complete line of CNC machining 
								cente

In [17]:
#executing the crawler function 

excel_path = 'batchwise_companies.xlsx'
sheet = 'batch4'
web_col = 2
index_col = 0
parent_col = 1
output_folder = 'Crawled'
from_website = 1
to_website = 5

web_crawler_all(excel_path,sheet, web_col, index_col, parent_col,output_folder, from_website, to_website)

#send mail 
sendemail(from_addr = 'krannertdcmme@gmail.com', to_addr_list = ['mihirbhatia999@gmail.com'],
          cc_addr_list = ['djindal@purdue.edu'],subject = 'Test mail',message = 'this is a test mail 2 using Python',
          login = 'krannertdcmme', password = 'Abc12345!', smtpserver = 'smtp.gmail.com:587')

   Serial No.                     Company Name  \
0           1  Drug Plastics and Glass company   
1           2                Holscher Products   
2           3              Kerkhoff Associates   
3           4                   Mondi Bags Usa   
4           5                     Oxford House   

                                 Website  Has a Website?\n(1=Yes, 0=No)  \
0                http://drugplastics.com                            NaN   
1         http://holscherproductsinc.com                            NaN   
2                http://kacomponents.com                            NaN   
3                      http://pvgard.com                            NaN   
4  http://oxfordhouse.org/userfiles/file                            NaN   

   Static/Dynamic Website?  \
0                      NaN   
1                      NaN   
2                      NaN   
3                      NaN   
4                      NaN   

   If no website- Is present on Facebook?\n(1=Yes, 0=No)  Type of C

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHA

http://www.drugplastics.com/company/contact-us/
http://www.drugplastics.com/?p=18
http://www.drugplastics.com/wp-content/uploads/2014/12/CorporateEntrance.jpg
http://www.drugplastics.com/?p=19
http://www.drugplastics.com/wp-content/uploads/2014/09/MS_Company.jpg
http://www.drugplastics.com/company/recommended-hotels/
http://www.drugplastics.com/?p=21
http://www.drugplastics.com/?p=23
http://www.drugplastics.com/wp-content/uploads/2014/10/AboutUs.jpg
http://www.drugplastics.com/?p=2674
http://www.drugplastics.com/wp-content/uploads/2014/09/Quality-300x200.jpg
http://www.drugplastics.com/?p=31
http://www.drugplastics.com/wp-content/uploads/2014/09/Sustainability-300x200.jpg
http://www.drugplastics.com/?p=10676
http://www.drugplastics.com/products/?page=1
http://www.drugplastics.com/products/?page=2
http://www.drugplastics.com/products/?page=3
http://www.drugplastics.com/products/?page=4
http://www.drugplastics.com/products/?page=5
http://www.drugplastics.com/?family=32r
http://www.drugpl

http://www.drugplastics.com/?p=18//www.drugplastics.com
http://www.drugplastics.com/?p=18//s.w.org
http://www.drugplastics.com/?p=8032
http://www.drugplastics.com/?p=21/wp-content/uploads/2013/07/iStock_000017191861Medium-300x199.jpg
http://www.drugplastics.com/products/?page=1//netdna.bootstrapcdn.com/font-awesome/4.0.3/css/font-awesome.css?ver=4.0.3
http://www.drugplastics.com/?family=16e
http://www.drugplastics.com/?family=60n
http://www.drugplastics.com/?family=54e
http://www.drugplastics.com/?family=40h
http://www.drugplastics.com/?family=24a
http://www.drugplastics.com/?family=29e
http://www.drugplastics.com/products/2/
http://www.drugplastics.com/?family=10f
http://www.drugplastics.com/?family=50d
http://www.drugplastics.com/?family=10p
http://www.drugplastics.com/?family=60m
http://www.drugplastics.com/?family=12d
http://www.drugplastics.com/?family=60h
http://www.drugplastics.com/?family=60u
http://www.drugplastics.com/?family=28a
http://www.drugplastics.com/?family=24f
http:/

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHA

...

http://holscherproductsinc.com/index.php/#top
http://holscherproductsinc.com/index.php/customer/account/forgotpassword/
http://holscherproductsinc.com/index.php/customer/account/login/fonts.googleapis.com/css?family=Open+Sans:400,600
http://holscherproductsinc.com/index.php/customer/account/login/fonts.googleapis.com/css?family=Roboto:400,700
http://holscherproductsinc.com/index.php/shop-by-series/100-series-hanger-poles-and-accessories.html
http://holscherproductsinc.com/index.php/shop-by-series/1000-series-feeders.html
http://holscherproductsinc.com/index.php/shop-by-series/200-series-miscellaneous-poles-and-baffles.html
http://holscherproductsinc.com/index.php/shop-by-series/300-series-deck-hardware.html
http://holscherproductsinc.com/index.php/shop-by-series/400-series-bird-bath-hardware.html
http://holscherproductsinc.com/index.php/shop-by-series/500-series-cages-and-feeders.html
http://holscherproductsinc.com/index.php/shop-by-series/600-series-miscellaneous-hardware.html
http://

http://holscherproductsinc.com/index.php/shop-by-series/100-series-hanger-poles-and-accessories.html?p=2
http://holscherproductsinc.com/index.php/shop-by-series/100-series-hanger-poles-and-accessories.html?p=3
http://holscherproductsinc.com/index.php/shop-by-series/100-series-hanger-poles-and-accessories.html?mode=list
http://holscherproductsinc.com/index.php/shop-by-series/100-series-hanger-poles-and-accessories.html?dir=desc&order=sku
http://holscherproductsinc.com/media/catalog/product/cache/1/small_image/135x/9df78eab33525d08d6e5fb8d27136e95/1/0/100_2_.jpg
http://holscherproductsinc.com/media/catalog/product/cache/1/small_image/135x/9df78eab33525d08d6e5fb8d27136e95/1/0/103_2.jpg
http://holscherproductsinc.com/media/catalog/product/cache/1/small_image/135x/9df78eab33525d08d6e5fb8d27136e95/1/0/104_2.jpg
http://holscherproductsinc.com/media/catalog/product/cache/1/small_image/135x/9df78eab33525d08d6e5fb8d27136e95/1/0/105_2.jpg
http://holscherproductsinc.com/media/catalog/product/cache

http://holscherproductsinc.com/media/catalog/product/cache/1/image/600x600/9df78eab33525d08d6e5fb8d27136e95/2/2/222bc_7.png
http://holscherproductsinc.com/media/catalog/product/cache/1/thumbnail/600x600/9df78eab33525d08d6e5fb8d27136e95/2/2/223b-8.png
http://holscherproductsinc.com/media/catalog/product/cache/1/thumbnail/600x600/9df78eab33525d08d6e5fb8d27136e95/2/2/222bc_7.png
http://holscherproductsinc.com/media/catalog/product/cache/1/image/308x308/9df78eab33525d08d6e5fb8d27136e95/2/2/223b-8.png
http://holscherproductsinc.com/media/catalog/product/cache/1/thumbnail/97x/9df78eab33525d08d6e5fb8d27136e95/2/2/223b-8.png
http://holscherproductsinc.com/media/catalog/product/cache/1/thumbnail/97x/9df78eab33525d08d6e5fb8d27136e95/2/2/222bc_7.png
http://holscherproductsinc.com/media/catalog/product/cache/1/image/600x600/9df78eab33525d08d6e5fb8d27136e95/2/2/223b-8-fb.png
http://holscherproductsinc.com/media/catalog/product/cache/1/thumbnail/600x600/9df78eab33525d08d6e5fb8d27136e95/2/2/223b-8-fb

No of links = 0


kacomponents.com
http://kacomponents.com
No of links = 1


No of links = 0


pvgard.com
http://pvgard.com
No of links = 1


No of links = 0


oxfordhouse.org
http://oxfordhouse.org/userfiles/file
No of links = 1


No of links = 0


All links from the websites indicated have been crawled
Output File :  Crawled
No. of websites scraped : 5


1